# p5.js in the Jupyter Notebook for custom interactive visualizations 

### By Jeremy Tuloup - [@jtpio](https://twitter.com/jtpio) - [jtp.io](https://jtp.io) - [github.com/jtpio](https://github.com/jtpio)

The goal of this notebook is to show how to create custom visualizations in a Jupyter Notebook.


## What is p5.js?

From the [official website](https://p5js.org):

> p5.js is a JS client-side library for creating graphic and interactive experiences, based on the core principles of Processing. 

> make coding accessible for artists, designers, educators, and beginners, and reinterprets this for today's web.

In this notebook I decided to use p5.js. However this can be adapted to other libraries if needed. I just like p5 for its simplicity and values.

## Running Javascript code in the Jupyter Notebook

The Jupyter Notebook, like the Python language itself, also comes with batteries included, such as cell magics. One of the cell magics is `%%javascript`, which lets the user run Javascript code directly in the browser.

As an example, we can define a variable and print its value:

In [ ]:
%%javascript

let myVar = 'test'
console.log(myVar)

Nothing is show on the page and it is normal. `console.log` prints in the Developer Tools console. However the notebook exposes the `element` variable in the Input cell to reference the Output cell.

In [ ]:
%%javascript

element.text('Hello')

## Setting up the libraries


Jupyter uses [require.js](http://requirejs.org/) under the hood, and the good thing is that we can use of it for our own purposes.

The following block states that we are using:
- p5.js: rendering on the canvas
- lodash: utility library with many high order functions

In [ ]:
%%javascript
require.config({
    paths: {
        'p5': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.0/p5.min',
        'lodash': 'https://cdnjs.cloudflare.com/ajax/libs/lodash.js/4.17.4/lodash.min'
    }
});

## Jupyter Widgets

![view-model](./img/WidgetModelView.png)

- Source: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html#Why-does-displaying-the-same-widget-twice-work?

We can for example define small modules relying on p5.js, that will in the end be assembled together to form the bigger picture. The function defined under `window` will be accessible anywhere in the notebook when using `%%javascript` cell magic, and even in the developer tools console.

In [ ]:
%%javascript 

window.defineModule = function (name, dependencies, module) {
    // force the recreation of the module
    // (when re-executing a cell for example)
    require.undef(name);
    
    define(name, dependencies, module);
};

In [ ]:
%%javascript

window.createSketchView = function (name, dependencies, module) {
    
    require.undef(name);
    
    define(name,
           ['@jupyter-widgets/base', 'p5', 'lodash'].concat(dependencies),
           (widgets, p5, _, ...deps) => {

        let modelName = `${name}Model`;
        let viewName = `${name}View`;
        
        let Model = widgets.DOMWidgetModel.extend({
            defaults: function () {
                return _.extend(widgets.DOMWidgetModel.prototype.defaults.call(this), {
                    _model_name: modelName,
                    _view_name: viewName,
                    _model_module: name,
                    _view_module: name,
                    _model_module_version: '0.1.0',
                    _view_module_version: '0.1.0',
                });
            }
        });
        
        let View = widgets.DOMWidgetView.extend({
            initialize: function () {
                // create the dom element to contain the p5.js canvas
                this.node = document.createElement('div');
                this.node.setAttribute('style', 'text-align: center;');
                this.el.appendChild(this.node);
            },

            render: function () {
                // pass the model as the last dependency so it can
                // be accessed in the sketch
                let sketch = module(...deps, this.model);
                setTimeout(() => {
                    this.sketch = new p5(sketch, this.node);
                }, 0);
            },

            remove: function () {
                // stop the existing sketch when the view is removed
                // so p5.js can cancel the animation frame callback and free up resources
                if (this.sketch) {
                    this.sketch.remove();
                    this.sketch = null;
                }
            }
        });
        
        return {
            [modelName] : Model,
            [viewName] : View,
        };
    });
}

The `defineModule` javascript function can now be used to create a module containing a bunch of parameters (for testing).

In [ ]:
%%javascript

// Test module defining a few constants, for example purposes
// Such constants should ideally be defined directly in the model
// and directly accessed by the view

defineModule('testModule', [], () => {
    const W = 500;
    const H = 500;
    
    return {W, H};
})

## 2D Demo

### Defining the view

In [ ]:
%%javascript

createSketchView('Sketch2D', ['testModule'], (TestModule, model) => {
    return function(p) {
        const {W, H} = TestModule;
        const [CX, CY] = [W / 2, H / 2];
        
        p.setup = function(){
            p.createCanvas(W, H);
            p.rectMode(p.CENTER);
        }

        p.draw = function () {
            p.background('#ddd');
            p.translate(CX, CY);
            let n = model.get('n_squares');
            _.range(n).forEach(i => {
                p.push();
                p.rotate(p.frameCount / 200 * (i + 1));
                p.fill(i * 5, i * 100, i * 150);
                p.rect(0, 0, 200, 200);
                p.pop();
            });
        }
    };
})

### Defining the model

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, Int, default


class Sketch2D(widgets.DOMWidget):
    _view_name = Unicode('Sketch2DView').tag(sync=True)
    _view_module = Unicode('Sketch2D').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    n_squares = Int(1).tag(sync=True)

In [ ]:
sketch_2d = Sketch2D()
sketch_2d

In [ ]:
sketch_2d.n_squares = 4

## 3D Demo

P5.js even supports basic 3D primitives, so let's create a new widget to test that!

In [ ]:
%%javascript

createSketchView('Sketch3D', ['testModule'], (Settings, model) => {
    return function(p) {
        const {W, H} = Settings;
        
        p.setup = function(){
            p.createCanvas(W, H, p.WEBGL);
        }

        p.draw = function () {
            p.background('#ddd');
            let t = p.frameCount;
            let n = model.get('n_cubes');
            p.randomSeed(42);
            _.range(n).forEach(i => {
                const R = 180// + 30 * p.sin(t * 0.2 + i);
                const x = R * p.cos(i * p.TWO_PI / n);
                const y = R * p.sin(i* p.TWO_PI / n);
                p.push();
                p.translate(x, y);
                p.fill(p.random(255), p.random(255), p.random(255));
                p.rotateY(t * 0.05 + i);
                p.box(50);
                p.pop();
            });
        } 
    };
})

In [ ]:
class Sketch3D(widgets.DOMWidget):
    _view_name = Unicode('Sketch3DView').tag(sync=True)
    _view_module = Unicode('Sketch3D').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    n_cubes = Int(4).tag(sync=True)

In [ ]:
sketch_3d = Sketch3D()
sketch_3d

In [ ]:
sketch_3d.n_cubes = 10

All of this is great! What about looking at a concrete example?

## Concrete example with the Bermuda Triangle Puzzle


The Bermuda Triangle Puzzle is a wooden puzzle consisting of 1 big triangle and 16 smaller triangles of equal size.

On the edges of the big triangle, there are colored dots, as well as on the edges of the small triangles:

![Bermuda Triangle Puzzle](./img/bermuda_triangle_puzzle.jpg)

## Goal

The 16 triangles all fit inside the big triangle. But the goal of the puzzle is to **arrange the triangles in a way so that each adjacent color is the same: red next to red, blue next to blue, and so on...**.

One thing to notice is that the colors on the edges of the big triangle cannot be moved, but the smaller triangles can be rotated and re-arranged as we want.

### How do we solve this?

As usual, there are different approaches:

1. One can just take the puzzle and start playing with it. After a while, some patterns may show up but can become very tedious when trying all combinations.
2. Speaking of trying all combinations, it is also possible to brute force the puzzle manually with the wooden pieces, take notes of the configurations that have been tested, and keep iterating. This requires a lot of rigor to not miss any step and start from the beginning.
3. Brute-forcing can be fun for a moment, but repetitive after a few minutes. Instead of doing it manually, we write a program that goes through all possibilities and try them one by one, maybe finding more than one solution if more exist. This requires knowing how to approach the problem and define an explicit and efficient search strategy, which can be difficult to find and implement.
4. When we don't really know what to do, we can start modelling the puzzle / state, and use stochastic search to find a solution. This requires a set of good building blocks and tools that the stochastic search can rely on. You can find examples of using Simulated Annealing on my [blog](https://jtp.io) to solve the [Aristotle Number Puzzle](https://jtp.io/2017/01/12/aristotle-number-puzzle.html) or [Nine-Color Cube](https://jtp.io/2017/05/10/nine-color-cube.html).

## Strategy

In this notebook, we will go with the third approach, using a recursive search.
To prove that a solution is correct, the best way is to draw it right away, and visually verify the colors match. This is what we would do with the real puzzle.

Since we will already be drawing the puzzle, we can push one step further to make an interactive animation of the search.

Steps:

1. Model the puzzle by encoding the colors to numbers and arranging them into lists.
2. Define a function, which draws a state on a canvas. A state is defined as the big triangle and a permutation of the 16 small triangles potentially rotated.
3. Define a function, which given a list of actions (permutations of triangles, rotation of triangles), animate them on a canvas.
5. Define a function that uses the helper to search for the solution, logging each step taken (permutation, rotation) to a list of actions.
6. Use the previously defined functions to animated the search and display the final solution.

## Model

This is perhaps the most tedious step, as it requires listing the different pieces with the proper order for the colors.

### Triangles

In total, we have 16 triangles, and 6 colors: **white, blue, yellow, green, black, red**.

In [ ]:
N_TRIANGLES = 16
IDS = list(range(N_TRIANGLES))
N_COLORS = 6
WHITE, BLUE, YELLOW, GREEN, BLACK, RED = range(N_COLORS)

# list the pieces, turning anti-clockwise for the colors
# The number at the last position of the tuple indicate the number
# of identical pieces (cf photo above)
triangles_count = [
    (WHITE, BLUE, BLUE, 1),
    (WHITE, YELLOW, GREEN, 2),  # 2 of these
    (WHITE, BLACK, BLUE, 2),  # 2 of these
    (WHITE, GREEN, RED, 1),
    (WHITE, RED, YELLOW, 1),
    (WHITE, WHITE, BLUE, 1),
    (BLACK, GREEN, RED, 1),
    (BLACK, RED, GREEN, 2),  # 2 of these
    (BLACK, BLACK, GREEN, 1),
    (BLACK, GREEN, YELLOW, 1),
    (BLACK, YELLOW, BLUE, 1),
    (GREEN, RED, YELLOW, 1),
    (BLUE, GREEN, YELLOW, 1)
]

assert N_TRIANGLES == sum(t[-1] for t in triangles_count)

The triangles were listed correctly, let's unpack them into a flat list.

In [ ]:
triangles = tuple([t[:-1] for t in triangles_count for times in range(t[-1])])
print(triangles)

assert N_TRIANGLES == len(triangles)

This is our immutable list of triangles (tuples), that we can index with an integer from 0 to 15.

### Board

What needs to be done now is to define the board with the fixed colors. Together with the triangles, they define the **state** of the puzzle.

In [ ]:
from traitlets import List, Tuple

class Board(widgets.DOMWidget):    
    TRIANGLES = List(triangles).tag(sync=True)
    LEFT = Tuple((WHITE, RED, WHITE, YELLOW)).tag(sync=True)
    RIGHT = Tuple((BLUE, RED, GREEN, BLACK)).tag(sync=True)
    BOTTOM = Tuple((GREEN, GREEN, WHITE, GREEN)).tag(sync=True)
    permutation = List([]).tag(sync=True)
    
    def random(self):
        self.permutation = [triangles[i] + (0,) for i in range(N_TRIANGLES)]

In [ ]:
Board(permutation=[[i, 0] for i in range(N_TRIANGLES)])

## Drawing the state

We can now reuse the previously defined Javascript helpers to create modules and sketches on demand.

In [ ]:
%%javascript

// define a list of constant such as the size of the base canvas,
// the size of the triangles, colors...
defineModule('settings', [], () => {
    const ANIM_W = 800;
    const ANIM_H = ANIM_W / 1.6;
    const N_TRIANGLES = 16;
    const COLORS = ['#fff', '#00f', '#ff0', '#0f0', '#000', '#f00'];
    const WOOD_COLOR = '#825201';
    const R = 50;
    const r = R / 2;
    const CR = r / 2;
    const OFFSET_X = R * Math.sqrt(3) * 0.5;
    const OFFSET_Y = 1.5 * R;
    
    return {ANIM_W, ANIM_H, N_TRIANGLES, WOOD_COLOR, COLORS, R, r, CR, OFFSET_X, OFFSET_Y};
})

With these parameters defined, we can create a `triangles` module that will be used to draw one triangle on the screen.

In [ ]:
%%javascript

defineModule('triangles', ['settings'], (settings) => {
    const {COLORS, WOOD_COLOR, R, r, CR, OFFSET_X, OFFSET_Y} = settings;
    
    function _getPoints(n, startAngle, radius) {
        let points = [];
        const da = 2 * Math.PI / n;
        for (let i = 0; i < n; i++) {
            const angle = startAngle - i * da;
            const x = radius * Math.cos(angle);
            const y = radius * Math.sin(angle);
            points.push(x);
            points.push(y);
        }
        return points;
    }
    
    return (p) => {
        return {
            getTrianglePoints: _getPoints,
            genTrianglesPos: function () {
                let positions = [];
                let id = 0;
                for (let row = 0; row < 4; row++) {
                    const n_row = 2 * row + 1;
                    for (let col = 0; col < n_row; col++) {
                        const x = (col - row) * OFFSET_X;
                        const flip = (id + row) % 2;
                        const y = row * OFFSET_Y + ((flip == 1) ? -R/2 : 0);
                        positions.push({id, x, y, flip, row, col, n_row});
                        id++;
                    }
                }
                return positions;
            },
            drawTriangle: function (colors, x, y, rotation, flip=0) {
                const n = colors.length;
                
                p.fill(WOOD_COLOR);
                p.push();
                p.translate(x, y);
                p.rotate(-rotation * p.TWO_PI / 3 + flip * p.PI);
                
                p.triangle(..._getPoints(n, Math.PI / 6, R));

                let circles = _getPoints(n, Math.PI / 2, 1.25 * CR);
                for (let i = 0; i < n; i++) {
                    const xx = circles[2*i];
                    const yy = circles[2*i+1];
                    const color = COLORS[colors[i]];
                    p.fill(color);
                    p.ellipse(xx, yy, CR);
                }
                p.pop();
            }
        };
    };
});

### Static Board

Let's start small and focus on drawing a static board. This is were the different modules defined above become useful, as we can now depend on them.

In [ ]:
%%javascript

defineModule('staticBoard', ['settings', 'triangles'], (Settings, Triangles) => {
    let {COLORS, R, CR, OFFSET_X, OFFSET_Y} = Settings;
    
    return (p) => {
        let triangles = Triangles(p);
        
        function _drawStaticColors (left, right, bottom) {
            for (let {id, x, y, row, col, n_row} of triangles.genTrianglesPos()) {
                if (col === 0) {
                    const colorLeft = COLORS[left[row]];
                    const colorRight = COLORS[right[row]];
                    p.fill(colorLeft);
                    p.ellipse(x - OFFSET_X, y - R / 2, CR);
                    p.fill(colorRight);
                    p.ellipse(x + n_row * OFFSET_X, y - R / 2, CR);
                }
                          
                if (row === 3 && col % 2 == 0) {
                    p.fill(COLORS[bottom[parseInt(col / 2, 10)]]);
                    p.ellipse(x, 3.75 * OFFSET_Y, CR);
                }
            }
        }
        
        function _drawFrame () {
            const positions = triangles.genTrianglesPos();
            const mid = positions[6];
            p.push();
            p.noFill();
            p.stroke(0);
            p.strokeWeight(2);
            p.translate(mid.x, mid.y);
            p.triangle(...triangles.getTrianglePoints(3, Math.PI / 6, 4 * R));
            p.pop();
        }
        
        return {
            drawStaticColors: _drawStaticColors,
            drawFrame: _drawFrame,
            drawTriangles: (permutation) => {
                for (let {id, x, y, flip} of triangles.genTrianglesPos()) {
                    let [a, b, c, rot] = permutation[id];
                    p.push();
                    triangles.drawTriangle([a, b, c], x, y, rot, flip);
                    p.pop();
                }
            }
        };
    };
});

Creating the sketch is now just a matter of putting the previous pieces together:

In [ ]:
%%javascript

createSketchView('StaticBoard', ['staticBoard'], (StaticBoard, model) => {
    return function(p) {
        const W = 400;
        const H = 400;
        const LEFT = model.get('LEFT');
        const RIGHT = model.get('RIGHT');
        const BOTTOM = model.get('BOTTOM');
        let staticBoard = StaticBoard(p);

        p.setup = function(){
            p.createCanvas(W, H);
        }

        p.draw = function () {
            p.background('#ddd');
            p.push();
            p.translate(W / 2, H / 4);
            let permutation = model.get('permutation');
            staticBoard.drawFrame();
            staticBoard.drawTriangles(permutation);
            staticBoard.drawStaticColors(LEFT, RIGHT, BOTTOM);
            p.pop();
        }
    };
})

In [ ]:
from random import sample
from traitlets import List

class StaticBoard(Board):
    _view_name = Unicode('StaticBoardView').tag(sync=True)
    _view_module = Unicode('StaticBoard').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _model_name = Unicode('StaticBoardModel').tag(sync=True)
    _model_module = Unicode('StaticBoard').tag(sync=True)
    _model_module_version = Unicode('0.1.0').tag(sync=True)
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.random()
    
    def shuffle(self):
        self.permutation = sample(self.permutation, N_TRIANGLES) 

In [ ]:
testStaticBoard = StaticBoard()
testStaticBoard

In [ ]:
testStaticBoard.shuffle()

We can see that all the pieces are drawn, but they are not placed at the correct position, which is normal since we haven't even tried to solve the problem yet!

But this is already looking good and this static visualization lets us verify quickly whether or not a solution is valid.

### Rotating the triangles

You must have noticed the magic `0` number as the last element of the triangle tuple when creating a permutation of the `Board`.

This number can take the values `[0, 1, 2]` and corresponds to the rotation of a triangle. To make it simple, we order the three colors of the triangles from 0 to 2, with 0 corresponding to the color at the "bottom" of the triangle.

Difficult to visualize? No problem, we can create a new sketch for that purpose!

Since we want to visualize the rotation of the triangle, let's add another library to our toolbet: [tween.js](https://github.com/tweenjs/tween.js/). With tween.js, we can create tweens which are convenient helpers to create interpolations between values. 

In [ ]:
%%javascript

require.config({
    paths: {
        tween: 'https://cdnjs.cloudflare.com/ajax/libs/tween.js/17.1.1/Tween.min'
    }
});

Now let's define a sketch showcasing the use of tween.js with our triangles.

In [ ]:
%%javascript

createSketchView('RotateDemo', ['tween', 'triangles'], (Tween, Triangles, model) => {
    const W = 300; const H = 150;
    
    return (p) => {
        let obj = { angle: 0 };
        let T = Triangles(p);
        let rotateButton;
        
        let tweenGroup = new Tween.Group();
        let t = new Tween.Tween(obj, tweenGroup)
                    .to({angle: "+" + (p.TWO_PI / 3)}, 500)
                    .easing(Tween.Easing.Quadratic.InOut)
                    .onStart(() => t.running = true)
                    .onComplete(() => t.running = false)
        
        function rotate () {
            if (t.running) return;
            t.start();
        }
        
        model.on('change:rotations', rotate);

        p.setup = function(){
            p.createCanvas(W, H);
        }

        p.draw = function () {
            tweenGroup.update();
            p.background('#ddd');
            p.translate(W / 3, H / 2);
            p.push();
            p.rotate(obj.angle);
            T.drawTriangle([0, 1, 2], 0);
            p.pop();
            p.push();
            p.translate(W / 3, 0);
            p.rotate(-obj.angle);
            T.drawTriangle([3, 4, 5], 0);
            p.pop();
        }
    };
});

In [ ]:
class RotateDemo(Board):
    _view_name = Unicode('RotateDemoView').tag(sync=True)
    _view_module = Unicode('RotateDemo').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _model_name = Unicode('RotateDemoModel').tag(sync=True)
    _model_module = Unicode('RotateDemo').tag(sync=True)
    _model_module_version = Unicode('0.1.0').tag(sync=True)
    rotations = Int(0).tag(sync=True)

In [ ]:
rotate_button = widgets.Button(description="Rotate the triangles")

def on_button_clicked(b):
    rotate_demo.rotations += 1

rotate_button.on_click(on_button_clicked)
rotate_demo = RotateDemo()

widgets.HBox([rotate_button, rotate_demo])

### Animated Board

As stated above, the goal is to also create an animation of the solving process.


Similar to the previous examples, we can define a new require module called `animatedBoard`.

In [ ]:
%%javascript

defineModule('animationUtils', ['settings', 'triangles', 'lodash'], (Settings, triangles, _) => {
    const {ANIM_W, N_TRIANGLES} = Settings;
    
    return {
        // positions around the big triangles (outside)
        outPositions: _.range(N_TRIANGLES).map(i => {
            return {
                x: ANIM_W * 0.3 + (i % 4) * 100,
                y: Math.floor(i / 4) * 100,
                r: 0,
                f: 0,
            };
        }),
        
        // positions of the small triangles inside the big triangle
        inPositions: triangles().genTrianglesPos().map(t => _.pick(t, ['x', 'y', 'flip']))
    };
});

In [ ]:
%%javascript

defineModule('animatedBoard',
             ['settings', 'animationUtils', 'staticBoard', 'triangles', 'tween', 'lodash'],
             (Settings, Utils, StaticBoard, Triangles, Tween, _) => {
    
    const {ANIM_W, ANIM_H, N_TRIANGLES} = Settings;
    
    return (p) => {
        let tweenGroup = new Tween.Group();
        let globalTime = 0;
        
        let staticBoard = StaticBoard(p);
        let triangles = Triangles(p);
        
        let states = model.get('states');
        const [out, pos] = [Utils.outPositions, Utils.inPositions];
        
        // arrays of positions to create the tweens (animations)
        let start = _.range(N_TRIANGLES).map(i => ({x: 0, y: 0, r: 0, f: 0}));
        let end = _.range(N_TRIANGLES).map(i => ({x: 0, y: 0, r: 0, f: 0}));
        
        // store the triangles moving at each turn to display them on top of the others
        let moving = [];
        
        function findPos(triangleId, state) {
            return state.findIndex(e => (e && e[0] === triangleId));
        }

        function transitionState(curr) {
            const [from, to] = [states[curr], states[curr + 1]];
            _.range(N_TRIANGLES).forEach(i => {
                
                const [startPos, endPos] = [findPos(i, from), findPos(i, to)];
                
                // on the board
                if (startPos > -1 && endPos > -1) {
                    _.assign(start[i], {x: pos[startPos].x, y: pos[startPos].y, r: from[startPos][1], f: pos[startPos].flip});
                    _.assign(end[i], {x: pos[endPos].x, y: pos[endPos].y, r: to[endPos][1], f: pos[endPos].flip});
                    return;
                }
                
                // not in current state but in the next one
                if (startPos < 0 && endPos > -1) {
                    _.assign(start[i], {x: out[i].x, y: out[i].y, r: out[i].r, f: out[i].f});
                    _.assign(end[i], {x: pos[endPos].x, y: pos[endPos].y, r: to[endPos][1], f: pos[endPos].flip});
                    return;
                }
                
                // in current state but not in the next one, bring back
                if (startPos > -1 && endPos < 0) {
                    _.assign(start[i], {x: pos[startPos].x, y: pos[startPos].y, r: from[startPos][1], f: pos[startPos].flip});
                    _.assign(end[i], {x: out[i].x, y: out[i].y, r: out[i].r, f: out[i].f});
                    return;
                }
                
                // out, no movement
                if (startPos < 0 && endPos < 0) {
                    _.assign(start[i], {x: out[i].x, y: out[i].y, r: out[i].r, f: out[i].f});
                    _.assign(end[i], start[i]);
                    return;
                }
            });

            moving = [];
            start.forEach((a, i) => {
                const b = end[i];
                if (a.x != b.x || a.y != b.y || a.r != b.r) {
                    moving.push(i);
                    new Tween.Tween(a, tweenGroup)
                        .to({x: b.x, y: b.y, r: b.r, f: b.f}, animationSpeed)
                        .easing(Tween.Easing.Quadratic.InOut)
                        .start(globalTime)
                }
            });
            
            new Tween.Tween({t: 0}, tweenGroup)
                .to({t: animationSpeed}, animationSpeed)
                .onComplete(() => {
                    if (it >= states.length - 2) return;
                    transitionState(curr + Math.min(step, states.length - it - 1));
                })
                .start(globalTime);
            
        }
        
        function setState(stateId) {
            tweenGroup.removeAll();
            it = Math.max(0, Math.min(stateId, board.states.length - 2));
            transitionState(it);
            running = false;
        }

        return {
            startAnimation: (startIteration, speed) => {
                it = startIteration;
                baseSpeed = speed;
                animationSpeed = speed;
                transitionState(0);
            },
            draw: () => {
                if (running) {
                    tweenGroup.update(globalTime);  
                    globalTime += Math.min(1000 / p.frameRate(), 33);
                }
                
                p.fill(0);
                p.textSize(24);
                p.text(`Iteration: ${it}`, 10, 30);
                
                p.translate(ANIM_W / 4, ANIM_H / 4);
                
                staticBoard.drawFrame();
                
                let allTriangles = _.range(N_TRIANGLES);
                let staticTriangles = _.difference(allTriangles, moving);
                [staticTriangles, moving].forEach(bucket => {
                    bucket.forEach(triangleId => {
                        const [a, b, c] = board.TRIANGLES[triangleId];
                        const {x, y, r, f} = start[triangleId];
                        p.push();
                        triangles.drawTriangle([a, b, c], x, y, r, f);
                        p.pop();
                    });
                });
                staticBoard.drawStaticColors(board);
            }
        };
    };
});

In [ ]:
%%javascript

createSketchView('AnimatedBoard', ['animatedBoard', 'settings'], (AnimatedBoard, Settings, model) => {
    const {ANIM_W, ANIM_H} = Settings;

    const W = Math.min(element[0].offsetWidth, 900); 
    const H = W / (ANIM_W / ANIM_H);

    return function(p) {
        let board = AnimatedBoard(p);

        p.setup = function () {
            let canvas = p.createCanvas(W, H);
            board.startAnimation(startIteration, speed);
        }

        p.draw = function () {
            p.scale(W / ANIM_W);
            p.background('#ddd');
            board.draw();
        }
    };
});

In [ ]:
class AnimatedBoard(Board):
    _view_name = Unicode('AnimatedBoardView').tag(sync=True)
    _view_module = Unicode('AnimatedBoard').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    _model_name = Unicode('AnimatedBoardModel').tag(sync=True)
    _model_module = Unicode('AnimatedBoard').tag(sync=True)
    _model_module_version = Unicode('0.1.0').tag(sync=True)
    
    states = List([]).tag(sync=True)
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

Let's create a test `AnimatedBoard` for testing.

In [ ]:
animated_board = AnimatedBoard(
    permutation=[[i, 0] for i in range(N_TRIANGLES)],
    states=[
        [None] * 16,
        [[7, 0]] + [None] * 15,
        [[7, 1]] + [None] * 15,
        [[7, 2]] + [None] * 15,
        [[7, 2], [0, 0]] + [None] * 14,
        [[7, 2], [0, 1]] + [None] * 14,
        [[7, 2], [0, 2]] + [None] * 14,
    ]
)
animated_board

In [ ]:
from ipywidgets import Layout, Button, Box, VBox, ToggleButton

items_layout = Layout(flex='flex-stretch', width='auto')

iteration_slider = widgets.IntSlider(max=10000, description='Iteration', layout=Layout(width='100%'))
prev_button = Button(description='Previous', button_style='info')
next_button = Button(description='Next', button_style='info')
play_button = ToggleButton(description='Play / Pause', button_style='info')

items = [prev_button, play_button, next_button, iteration_slider]

box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', border='solid', width='100%')
box = VBox([Box(children=items, layout=box_layout)])
box

## Recursive Search

### Helper functions

In [ ]:
import numpy as np

cumsum = np.cumsum([2 * k + 1 for k in range(4)])


def pack(j, i, c):
    return sum(2 * k + 1 for k in range(j)) + i

def tri(j, i, c):
    n = pack(j, i, c)
    return f'triangles[p[{n}][0]][{c}-p[{n}][1]]'

def unpack(triangle_id):
    j = next(i for i, s in enumerate(cumsum) if s > triangle_id)
    i = 2 * j + 1
    k = max(0, triangle_id - cumsum[max(0, j - 1)] )
    return (j, i, k)

A board is valid if there is not conflicting colors. Some slots can be missing (marked as `None`, meaning not placed yet).

In [ ]:
def generate_valid_triangle(triangle_id, reverse=False):
    j, i, k = unpack(triangle_id)
    code = []
    if k == 0:
        # on the left edge
        code.append(f'{tri(j, k, 2)} == Board.LEFT[{j}]')

    if k == i - 1:
        # on the right edge
        code.append(f'{tri(j, k, 1)} == Board.RIGHT[{j}]')

    if j == 3 and k % 2 == 0:
        # on the bottom edge
        code.append(f'{tri(j, k, 0)} == Board.BOTTOM[{k//2}]')

    if k % 2 == 0:
        # normal orientation (facing up)
        if j < 3:
            # match with line below
            code.append(f'(not p[{pack(j + 1, k + 1, 0)}] or {tri(j, k, 0)} == {tri(j + 1, k + 1, 0)})')
        if k > 0:
            # match with triangle on the left
            code.append(f'(not p[{pack(j, k - 1, 2)}] or {tri(j, k, 2)} == {tri(j, k - 1, 2)})')
        if k < i - 1:
            # match with triangle on the right
            code.append(f'(not p[{pack(j, k + 1, 1)}] or {tri(j, k, 1)} == {tri(j, k + 1, 1)})')
            
    if k % 2 == 1 and reverse:
        # reverse orientation (facing down)
        # match with line above
        code.append(f'(not p[{pack(j - 1, k - 1, 0)}] or {tri(j, k, 0)} == {tri(j - 1, k - 1, 0)})')
        # match with triangle on the left
        code.append(f'(not p[{pack(j, k - 1, 2)}] or {tri(j, k, 2)} == {tri(j, k - 1, 2)})')
        # match with triangle on the right
        code.append(f'(not p[{pack(j, k + 1, 1)}] or {tri(j, k, 1)} == {tri(j, k + 1, 1)})')
        
    return code

generate_valid_triangle(0, False)

In [ ]:
def generate_valid_function():
    code = []
    for n in range(N_TRIANGLES):
        _, _, k = unpack(n)
        if k % 2 != 0:
            continue
        code += [f"(not p[{n}] or ({' and '.join(generate_valid_triangle(n))}))"]
    return 'def is_valid (p):\n\treturn ' + ' and '.join(code) + '\n\t'


code = generate_valid_function()
print('Code:')
print(code)
print('Evaluating the code...')
exec(code)
assert 'is_valid' in globals()
print('done.')

In [ ]:
state = [None for i in range(N_TRIANGLES)]
state[0] = [3, 0]
print(state)
assert not is_valid(state)

state[0] = [0, 2]
print(state)
assert is_valid(state)

In [ ]:
from copy import deepcopy


class RecursiveSolver():
    '''
    Usage
    -----
    
    solver = RecursiveSolver()
    solver.run()

    solver.board    # state of the board at the end of execution
    solver.found()  # returns True if a solution was found 
    '''
    
    def __init__(self):
        self.reset_state()
        
    def reset_state(self):
        self.board = [None] * N_TRIANGLES
        self.used = [0] * N_TRIANGLES
        self.log = [deepcopy(self.board)]
        self.it = 0
        
    def _log(self):
        self.log.append(deepcopy(self.board))
        
    def _place(self, i):
        self.it += 1
        if i == N_TRIANGLES:
            return True
        
        for j in range(N_TRIANGLES - 1, -1, -1):
            if self.used[j]:
                # piece number j already used
                continue
                
            self.used[j] = 1
            
            for rot in range(3):
                # place the piece on the board
                self.board[i] = (j, rot)
                self._log()

                # stop the recursion if the current configuration
                # is not valid or a solution has been found
                if is_valid(self.board) and self._place(i + 1):
                    return True

            # remove the piece from the board
            self.board[i] = None
            self._log()
                
            self.used[j] = 0
            
        return False

    def run(self):
        self.reset_state()
        self._place(0)
        return self.board
    
    def found(self):
        return all(slot is not None for slot in self.board)

In [ ]:
%%time

solver = RecursiveSolver()
res = solver.run()
if solver.found():
    print('Solution found')
    print(res)
    print(f'{len(solver.log)} actions')
    send_to_js('recursive', Board(res, solver.log))
else:
    print('No solution found')

A solution was found!

We can also double check the first states of the log that will be used for the animation.

In [ ]:
for s in solver.log[:10]:
    print(s)

In [ ]:
%%javascript

drawStaticBoard('solution-recursive', 'recursive', element)

In [ ]:
%%javascript

animateBoard('animation-recursive', 'recursive', element, 0, 2500)

And with the real physical pieces:

![Real Puzzle Solution](./img/bermuda_triangle_solved.jpg)

## Summary

In this notebook, we managed to combine all the core strengths of Javascript, Python and the Jupyter Notebook to create a custom and interactive walkthrough of the resolution of a puzzle.

The core logic and the resolution of the problem can be implemented in Python. On the other hand the rendering and visual feedback can be delegated to Javascript (which excels at it), making once again the best out of the tools at our disposition.
Finally, the Jupyter Notebook acts as the glue between the two worlds.

There are however a couple of things that can be improved for this workflow that I am going to list below without particular order:

- Transfering data from Python to Javascript (in our case the JSON blob sent from Python to Javascript) is not convenient and has to go through the use of the "shared global `window` memory. Instead it would make more sense to implement a [Jupyter Widget](https://github.com/jupyter-widgets/ipywidgets) to benefit from the underlying Websockets communication at the core of the notebook itself. With a widget, all the code can be written in Python and the Javascript details are abstracted away.
- The setup code at the beginning of the notebook and the animations were tailored for this specific problem and not really designed with reusability in mind. One could built a mini-libraries of such helper functions on the side that can be imported in the notebook (as Python packages or by executing the content of a Javascript file).
- Does it work in JupyterLab? At the moment of writing it's a no (sadly), as arbitrary execution of Javascript is prohibited. I haven't looked into this in details yet but there surely will be (or already is) a way to integrate with Javascript libraries as JupyterLab extensions.

## References and going further

I underwent quite a lot of research to find an existing library that would fit my needs (low level rendering to draw custom shapes, with interactivity). There are already some projects implementing similar use cases, but all still specific in some way:

- [Jake VanderPlas: The Python Visualization Landscape PyCon 2017](https://www.youtube.com/watch?v=FytuB8nFHPQ)
- Drawing from the presentation made by Nicolas P. Rougier: https://github.com/rougier/python-visualization-landscape
- [pythreejs](https://github.com/jovyan/pythreejs): Implemented as an Jupyter Widget
- [bqplot](https://github.com/bloomberg/bqplot): Great library for interactive data exploration
- [matplotlib patches](https://matplotlib.org/gallery/api/patch_collection.html?highlight=circles%20wedges%20polygons): Draw primitive shapes
- Let's create a **pyp5** Python package?  `¯\_(ツ)_/¯`